In [4]:
# Предобработка данных для датасета про золото

import pandas as pd
import numpy as np

# 1. Знакомство с данными
# Загружаем датасет
df = pd.read_csv('gold.csv')

# Первые 5 строк
print('Первые 5 строк:')
print(df.head())

# Последние 5 строк
print('\nПоследние 5 строк:')
print(df.tail())

# Случайная выборка 5 строк
print('\nСлучайная выборка:')
print(df.sample(5, random_state=42))

# Атрибуты датафрейма
print('\nФорма датафрейма (строки, столбцы):', df.shape)
print('Названия столбцов:', list(df.columns))

# Информация о датафрейме
print('\nИнформация о датафрейме:')
df.info()

# Использование памяти
print('\nИспользование памяти по столбцам:')
print(df.memory_usage(deep=True))

Первые 5 строк:
         Date   Open   High    Low  Close  Volume Currency
0  2000-01-04  289.5  289.5  280.0  283.7   21621      USD
1  2000-01-05  283.7  285.0  281.0  282.1   25448      USD
2  2000-01-06  281.6  282.8  280.2  282.4   19055      USD
3  2000-01-07  282.5  284.5  282.0  282.9   11266      USD
4  2000-01-10  282.4  283.9  281.8  282.7   30603      USD

Последние 5 строк:
            Date    Open     High      Low   Close  Volume Currency
5698  2022-08-29  1748.4  1757.90  1731.40  1749.7  156220      USD
5699  2022-08-30  1749.8  1752.80  1732.90  1736.3  130664      USD
5700  2022-08-31  1735.5  1738.00  1720.60  1726.2  176731      USD
5701  2022-09-01  1723.0  1723.00  1699.10  1709.3  198618      USD
5702  2022-09-02  1707.9  1729.45  1707.05  1723.0       0      USD

Случайная выборка:
            Date    Open    High     Low   Close  Volume Currency
1436  2005-10-04   469.0   472.7   467.4   469.3   43497      USD
748   2003-01-03   346.0   353.5   343.6   351.6  

In [5]:
# 2. Выявление проблем в данных
# Получим описательную статистику по числовым и категориальным столбцам
print('\nОписательная статистика (числовые столбцы):')
print(df.describe())
print('\nОписательная статистика (все столбцы):')
print(df.describe(include='all'))

# Проверим уникальные значения для категориальных признаков
print('\nУникальные значения для категориальных столбцов:')
for col in df.select_dtypes(include='object').columns:
    print(f'{col}:', df[col].unique()[:10])  # Первые 10 уникальных значений

# Здесь можно сделать выводы о структуре данных и выявленных проблемах (например, пропуски, выбросы, некорректные значения)
# Выводы пишите после анализа вывода выше



Описательная статистика (числовые столбцы):
              Open         High          Low        Close         Volume
count  5703.000000  5703.000000  5703.000000  5703.000000    5703.000000
mean   1040.382816  1048.339181  1031.863169  1040.298282  139141.669297
std     518.733377   522.353946   514.455903   518.524020  102537.449058
min     256.600000   259.400000   255.100000   256.600000       0.000000
25%     459.850000   463.900000   457.450000   460.500000   52938.500000
50%    1188.800000  1198.000000  1179.700000  1188.700000  126006.000000
75%    1381.400000  1392.750000  1368.100000  1383.050000  193109.000000
max    2076.400000  2089.200000  2049.000000  2069.400000  816531.000000

Описательная статистика (все столбцы):
              Date         Open         High          Low        Close  \
count         5703  5703.000000  5703.000000  5703.000000  5703.000000   
unique        5703          NaN          NaN          NaN          NaN   
top     2022-09-02          NaN     

**Выводы:**

Данные хорошо структурированы, большинство признаков числовые.
Столбец Date нужно привести к типу datetime.
Явных пропусков и дубликатов нет.

In [6]:
# 3. Проверка и приведение наименований столбцов и текста к нижнему регистру
# Приведём названия столбцов к змеиному регистру (snake_case) и нижнему регистру
print('\nПреобразуем названия столбцов к змеиному регистру и нижнему регистру:')
df.columns = [col.strip().replace(' ', '_').replace('-', '_').lower() for col in df.columns]
print('Новые названия столбцов:', list(df.columns))

# Приведём все текстовые значения к нижнему регистру
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.lower()
print('Все текстовые значения приведены к нижнему регистру.')




Преобразуем названия столбцов к змеиному регистру и нижнему регистру:
Новые названия столбцов: ['date', 'open', 'high', 'low', 'close', 'volume', 'currency']
Все текстовые значения приведены к нижнему регистру.


In [7]:
# 4. Проверка датафрейма на пропуски и их заполнение
print('\nПроверка пропусков:')
missing = df.isnull().sum()
print(missing[missing > 0])

# Заполняем пропуски: для числовых — медианой, для категориальных — модой
for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype == 'object':
            # Для категориальных признаков используем моду (самое частое значение)
            mode_value = df[col].mode()[0]
            df[col] = df[col].fillna(mode_value)
            print(f'Столбец {col}: пропуски заполнены модой ({mode_value})')
        else:
            # Для числовых признаков используем медиану
            median_value = df[col].median()
            df[col] = df[col].fillna(median_value)
            print(f'Столбец {col}: пропуски заполнены медианой ({median_value})')

# Если пропусков <5% и заполнить невозможно — удаляем такие строки
threshold = 0.05 * len(df)
for col in df.columns:
    if df[col].isnull().sum() > 0 and df[col].isnull().sum() <= threshold:
        df = df[df[col].notnull()]
        print(f'Столбец {col}: строки с пропусками удалены (менее 5% от общего числа)')




Проверка пропусков:
Series([], dtype: int64)


In [8]:
# 5. Изменение типов данных
print('\nИзменение типов данных, если необходимо:')
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            # Пробуем преобразовать к дате
            df[col] = pd.to_datetime(df[col])
            print(f'Столбец {col}: преобразован в datetime')
        except:
            try:
                # Пробуем преобразовать к float
                df[col] = df[col].astype(float)
                print(f'Столбец {col}: преобразован в float')
            except:
                # Оставляем как есть (категориальный)
                pass
# Объяснения: преобразуем к числовым или датам для корректной работы с данными и анализа




Изменение типов данных, если необходимо:
Столбец date: преобразован в datetime


In [9]:
# 6. Поиск и удаление дубликатов
print('\nПоиск дубликатов:')
dups = df.duplicated()
print(f'Количество дубликатов: {dups.sum()}')
if dups.sum() > 0:
    df = df.drop_duplicates()
    print('Дубликаты удалены.')
else:
    print('Дубликаты отсутствуют.')



Поиск дубликатов:
Количество дубликатов: 0
Дубликаты отсутствуют.


In [10]:
# 7. Промежуточные выводы
print('\nПромежуточные выводы:')
print(f'Итоговая форма датафрейма: {df.shape}')
print('Типы данных после обработки:')
print(df.dtypes)
print('Количество пропусков после обработки:')
print(df.isnull().sum().sum())
print('Предобработка завершена. Данные готовы к дальнейшему анализу.')


Промежуточные выводы:
Итоговая форма датафрейма: (5703, 7)
Типы данных после обработки:
date        datetime64[ns]
open               float64
high               float64
low                float64
close              float64
volume               int64
currency            object
dtype: object
Количество пропусков после обработки:
0
Предобработка завершена. Данные готовы к дальнейшему анализу.
